In [210]:
import pandas as pd
from rdflib import Graph
from pathlib import Path


# EDA

### Chargement des CSV (Philadelphia)

In [211]:
df_mode = pd.read_csv(
    "../data/Average_Daily_Ridership_By_Mode - City of Philadelphia.csv"
)

df_mode["date"] = pd.to_datetime(
    df_mode["Calendar_Year"].astype(str) + "-" +
    df_mode["Calendar_Month"].astype(str) + "-01"
)

df_mode = df_mode.rename(columns={
    "Mode": "mode",
    "Average_Daily_Ridership": "ridership"
})

df_mode["route_id"] = None
df_mode["level"] = "MODE"
df_mode["city"] = "Philadelphia"

df_mode = df_mode[
    ["city", "date", "route_id", "mode", "level", "ridership"]
]

df_mode.info()
df_mode.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   city       492 non-null    object        
 1   date       492 non-null    datetime64[ns]
 2   route_id   0 non-null      object        
 3   mode       492 non-null    object        
 4   level      492 non-null    object        
 5   ridership  492 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 23.2+ KB


,city,date,route_id,mode,level,ridership
0,Philadelphia,2019-01-01,None,Bus,MODE,459160
1,Philadelphia,2019-01-01,None,CCT,MODE,4294
2,Philadelphia,2019-01-01,None,Heavy Rail,MODE,296709
3,Philadelphia,2019-01-01,None,Regional Rail,MODE,122856
4,Philadelphia,2019-01-01,None,Trackless Trolley,MODE,16853


In [212]:
# Ridership par route

df_route = pd.read_csv(
    "../data/Average_Daily_Ridership_By_Route - City of Philadelphia.csv"
)

df_route["date"] = pd.to_datetime(
    df_route["Calendar_Year"].astype(str) + "-" +
    df_route["Calendar_Month"].astype(str) + "-01"
)

df_route = df_route.rename(columns={
    "Route": "route_id",
    "Average_Daily_Ridership": "ridership"
})

df_route["mode"] = None
df_route["level"] = "ROUTE"
df_route["city"] = "Philadelphia"

df_route = df_route[
    ["city", "date", "route_id", "mode", "level", "ridership"]
]

df_route.info()
df_route.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10994 entries, 0 to 10993
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   city       10994 non-null  object        
 1   date       10994 non-null  datetime64[ns]
 2   route_id   10994 non-null  object        
 3   mode       0 non-null      object        
 4   level      10994 non-null  object        
 5   ridership  10994 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 515.5+ KB


,city,date,route_id,mode,level,ridership
0,Philadelphia,2019-08-01,59,None,ROUTE,3811
1,Philadelphia,2019-08-01,60,None,ROUTE,8550
2,Philadelphia,2019-08-01,61,None,ROUTE,3563
3,Philadelphia,2019-08-01,62,None,ROUTE,226
4,Philadelphia,2019-08-01,64,None,ROUTE,4335


### Chargement des RDF (Chicago)

In [213]:
def rdf_daily_to_df(file_path):
    g = Graph()
    g.parse(file_path, format="xml")

    rows = {}

    for s, p, o in g:
        s = str(s)
        p = str(p)
        o = str(o)

        if s not in rows:
            rows[s] = {
                "subject": s,
                "city": "Chicago"
            }

        col = p.split("/")[-1]

        if col == "route":
            rows[s]["route"] = o
        elif col == "date":
            rows[s]["date"] = o
        elif col == "daytype":
            rows[s]["daytype"] = o
        elif col == "rides":
            rows[s]["ridership"] = o

    df = pd.DataFrame(rows.values())

    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["ridership"] = pd.to_numeric(df["ridership"], errors="coerce")

    return df


In [214]:
df_Daily = rdf_daily_to_df("../data/CTA Chicago - Ridership - Bus Routes - Daily Type Averages & Totals (RDF).rdf")
df_Daily.info()
df_Daily.shape
df_Daily.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   subject    500 non-null    object        
 1   city       500 non-null    object        
 2   ridership  500 non-null    int64         
 3   daytype    500 non-null    object        
 4   date       500 non-null    datetime64[ns]
 5   route      500 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 23.6+ KB


,subject,city,ridership,daytype,date,route
0,https://data.cityofchicago.org/resource/jyb9-n...,Chicago,1287,W,2001-01-04,68
1,https://data.cityofchicago.org/resource/jyb9-n...,Chicago,3035,W,2001-01-02,157
2,https://data.cityofchicago.org/resource/jyb9-n...,Chicago,1039,W,2001-01-03,122
3,https://data.cityofchicago.org/resource/jyb9-n...,Chicago,311,U,2001-01-01,43
4,https://data.cityofchicago.org/resource/jyb9-n...,Chicago,11313,W,2001-01-04,67


In [215]:
def rdf_monthly_chicago_to_df(file_path):
    g = Graph()
    g.parse(file_path, format="xml")

    data = {}

    for s, p, o in g:
        s = str(s)
        p = str(p)
        o = str(o)

        if s not in data:
            data[s] = {
                "subject": s,
                "city": "Chicago"
            }

        field = p.split("/")[-1]

        if field in [
            "route",
            "routename",
            "month_beginning",
            "avg_weekday_rides",
            "avg_saturday_rides",
            "avg_sunday_holiday_rides",
            "monthtotal"
        ]:
            data[s][field] = o

    df = pd.DataFrame(data.values())

    df["month_beginning"] = pd.to_datetime(df["month_beginning"], errors="coerce")

    numeric_cols = [
        "avg_weekday_rides",
        "avg_saturday_rides",
        "avg_sunday_holiday_rides",
        "monthtotal"
    ]

    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    return df



In [216]:
df_Monthly = rdf_monthly_chicago_to_df("../data/CTA Chicago - Ridership - Bus Routes - Monthly Day-Type Averages & Totals (RDF).rdf")
df_Monthly.head()

,subject,city,avg_weekday_rides,monthtotal,avg_sunday_holiday_rides,month_beginning,route,avg_saturday_rides,routename
0,https://data.cityofchicago.org/resource/bynn-g...,Chicago,7150.3,191162,3558.3,2001-04-01,71,5803.4,71st/South Shore
1,https://data.cityofchicago.org/resource/bynn-g...,Chicago,21175.6,500916,7245.6,2001-02-01,82,12105.2,Kimball-Homan
2,https://data.cityofchicago.org/resource/bynn-g...,Chicago,245.9,5410,0.0,2001-01-01,127,0.0,Madison/Roosevelt Circulator
3,https://data.cityofchicago.org/resource/bynn-g...,Chicago,531.9,10638,0.0,2001-02-01,17,0.0,Westchester
4,https://data.cityofchicago.org/resource/bynn-g...,Chicago,6445.2,169595,2187.6,2001-03-01,52A,3810.1,South Kedzie


In [217]:
df_Monthly.info()
df_Monthly.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   subject                   500 non-null    object        
 1   city                      500 non-null    object        
 2   avg_weekday_rides         500 non-null    float64       
 3   monthtotal                500 non-null    int64         
 4   avg_sunday_holiday_rides  500 non-null    float64       
 5   month_beginning           500 non-null    datetime64[ns]
 6   route                     500 non-null    object        
 7   avg_saturday_rides        500 non-null    float64       
 8   routename                 500 non-null    object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 35.3+ KB


(500, 9)

In [218]:
df_Daily = df_Daily.drop(columns=['subject', 'city'])
df_Monthly = df_Monthly.drop(columns=['subject', 'city'])

# Nettoyage


### Merge df_Daily et df_Monthly

In [219]:
# Merge des deux datasets df_Daily et df_Monthly

df_Daily['date'] = pd.to_datetime(df_Daily['date'], errors='coerce')
df_Daily['month_beginning'] = df_Daily['date'].dt.to_period('M').dt.to_timestamp()

df_Monthly['month_beginning'] = pd.to_datetime(df_Monthly['month_beginning'], errors='coerce')

df_merged = pd.merge(
    df_Daily,
    df_Monthly,
    on=['route', 'month_beginning'],
    how='left'
)
df_merged.to_csv("../data/chicago/Chicago_Transit_Authority_Bus_Ridership.csv", index=False)

In [220]:
df_merged.info()
df_merged.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ridership                 500 non-null    int64         
 1   daytype                   500 non-null    object        
 2   date                      500 non-null    datetime64[ns]
 3   route                     500 non-null    object        
 4   month_beginning           500 non-null    datetime64[ns]
 5   avg_weekday_rides         499 non-null    float64       
 6   monthtotal                499 non-null    float64       
 7   avg_sunday_holiday_rides  499 non-null    float64       
 8   avg_saturday_rides        499 non-null    float64       
 9   routename                 499 non-null    object        
dtypes: datetime64[ns](2), float64(4), int64(1), object(3)
memory usage: 39.2+ KB


,ridership,daytype,date,route,month_beginning,avg_weekday_rides,monthtotal,avg_sunday_holiday_rides,avg_saturday_rides,routename
0,1287,W,2001-01-04,68,2001-01-01,1579.5,38614.0,286.5,608.1,Northwest Highway
1,3035,W,2001-01-02,157,2001-01-01,3315.2,72933.0,0.0,0.0,Streeterville/Taylor
2,1039,W,2001-01-03,122,2001-01-01,1122.9,24705.0,0.0,0.0,Illinois Center/Ogilvie Express
3,311,U,2001-01-01,43,2001-01-01,1351.0,34213.0,354.8,679.3,43rd
4,11313,W,2001-01-04,67,2001-01-01,14219.5,387336.0,7025.6,9844.6,67th-69th-71st


In [221]:
df_merged.isnull().sum()

ridership                   0
daytype                     0
date                        0
route                       0
month_beginning             0
avg_weekday_rides           1
monthtotal                  1
avg_sunday_holiday_rides    1
avg_saturday_rides          1
routename                   1
dtype: int64

### Fusion Philadelphia df_mode et df_route

In [222]:
df_philadelphia = pd.concat([df_mode, df_route], ignore_index=True)

df_philadelphia.head()

df_philadelphia.to_csv(
   "../data/philadelphia/ridership_philadelphia_final.csv",
    index=False
)
df_philadelphia.info()
df_philadelphia.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11486 entries, 0 to 11485
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   city       11486 non-null  object        
 1   date       11486 non-null  datetime64[ns]
 2   route_id   10994 non-null  object        
 3   mode       492 non-null    object        
 4   level      11486 non-null  object        
 5   ridership  11486 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 538.5+ KB


,city,date,route_id,mode,level,ridership
0,Philadelphia,2019-01-01,None,Bus,MODE,459160
1,Philadelphia,2019-01-01,None,CCT,MODE,4294
2,Philadelphia,2019-01-01,None,Heavy Rail,MODE,296709
3,Philadelphia,2019-01-01,None,Regional Rail,MODE,122856
4,Philadelphia,2019-01-01,None,Trackless Trolley,MODE,16853


In [223]:
df_philadelphia = df_philadelphia.drop(columns=['city'])

In [224]:
df_philadelphia.isna().sum()

df_philadelphia["level"].value_counts()

df_philadelphia.groupby("level")[["route_id", "mode"]].nunique()


,route_id,mode
level,,
MODE,0,6
ROUTE,141,0
